In [45]:
%load_ext autoreload
%autoreload 2

import os; import sys; from pathlib import Path
root = Path(os.getcwd()).parents[0]
sys.path.append(str(root))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import joblib
import pandas as pd
from src.model import Regressor


In [47]:
scaler = joblib.load('../data/scaler.pkl') 
top_X_location = joblib.load('../data/top_X_location.pkl')
ohe = joblib.load('../data/location_one_hot_encoder.pkl')
model = Regressor().load_from_checkpoint('../model_checkpoints/epoch=35-avg_rmse_val=7.43.ckpt')


In [48]:
top_X_location

Index(['vasastan', 'södermalm', 'kungsholmen', 'östermalm', 'gärdet'], dtype='object')

In [58]:
df = pd.read_csv('../data/booli_10_pages_raw_20210506.csv')

display(
    df.head(3),
    f'shape = {df.shape}'
)

,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Bostadstyp,Driftskostnad,Våning,Byggår,Energiklass,BRF,adress,area_and_n_rooms,location,url,Avgift
0,5 200 000 kr,4 795 000 kr,+405 000 kr (+8.4 %),2021-05-05,148 600 kr/m²,12,Lägenhet,766 kr/mån,BV,1883.0,4,BRF Havssvalget 24,Skeppar­gatan 21,"35 m², 2 rum","Lägenhet, Östermalm",https://www.booli.se/bostad/732615,NaN
1,8 000 000 kr,6 695 000 kr,+1 305 000 kr (+19.5 %),2021-05-06,115 900 kr/m²,6,Lägenhet,599 kr/mån,2 tr,2019.0,NaN,BRF Tegeludden 16,Östhammars­gatan 70B,"69 m², 3 rum","Lägenhet, Gärdet",https://www.booli.se/bostad/4099014,3 876 kr/mån
2,5 010 000 kr,4 495 000 kr,+515 000 kr (+11.5 %),2021-05-05,106 600 kr/m²,12,Lägenhet,380 kr/mån,3 tr,1934.0,NaN,BRF Bojen 11,Kungs­holms Strand 141,"47 m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4200730,2 123 kr/mån


'shape = (283, 17)'

In [56]:
df['location'] = df['location'].str.replace('Lägenhet, ', '').str.lower()
df.loc[~df['location'].isin(top_X_location), 'location'] = 'other'
dummies = ohe.transform(df[['location']])
df = df.join(dummies)

In [57]:
df

,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Bostadstyp,Driftskostnad,Våning,Byggår,...,area_and_n_rooms,location,url,Avgift,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
0,5 200 000 kr,4 795 000 kr,+405 000 kr (+8.4 %),2021-05-05,148 600 kr/m²,12,Lägenhet,766 kr/mån,BV,1883.0,...,"35 m², 2 rum",östermalm,https://www.booli.se/bostad/732615,NaN,1,0,0,0,0,0
1,8 000 000 kr,6 695 000 kr,+1 305 000 kr (+19.5 %),2021-05-06,115 900 kr/m²,6,Lägenhet,599 kr/mån,2 tr,2019.0,...,"69 m², 3 rum",gärdet,https://www.booli.se/bostad/4099014,3 876 kr/mån,0,1,0,0,0,0
2,5 010 000 kr,4 495 000 kr,+515 000 kr (+11.5 %),2021-05-05,106 600 kr/m²,12,Lägenhet,380 kr/mån,3 tr,1934.0,...,"47 m², 2 rum",kungsholmen,https://www.booli.se/annons/4200730,2 123 kr/mån,0,0,1,0,0,0
3,7 550 000 kr,5 975 000 kr,+1 575 000 kr (+26.4 %),2021-05-05,109 400 kr/m²,12,Lägenhet,500 kr/mån,3 tr,1925.0,...,"69 m², 2½ rum",vasastan,https://www.booli.se/annons/4200566,3 423 kr/mån,0,0,0,1,0,0
4,7 150 000 kr,6 995 000 kr,+155 000 kr (+2.2 %),2021-05-06,94 100 kr/m²,12,Lägenhet,NaN,1 tr,1986.0,...,"76 m², 3 rum",other,https://www.booli.se/bostad/693186,3 544 kr/mån,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,3 100 000 kr,NaN,NaN,2021-04-28,77 500 kr/m²,NaN,Lägenhet,5 000 kr/mån,2 tr,1935.0,...,"40 m², 2 rum",other,https://www.booli.se/bostad/644549,2 847 kr/mån,0,0,0,0,1,0
279,5 575 000 kr,4 850 000 kr,+725 000 kr (+14.9 %),2021-04-28,111 500 kr/m²,4,Lägenhet,NaN,4 tr,1932.0,...,"50 m², 2 rum",östermalm,https://www.booli.se/annons/4202386,2 836 kr/mån,1,0,0,0,0,0
280,6 950 000 kr,6 395 000 kr,+555 000 kr (+8.7 %),2021-04-28,89 700 kr/m²,12,Lägenhet,440 kr/mån,7 tr,NaN,...,"77½ m², 3 rum",kungsholmen,https://www.booli.se/annons/4190871,3 257 kr/mån,0,0,1,0,0,0
281,2 760 000 kr,2 475 000 kr,+285 000 kr (+11.5 %),2021-04-28,115 000 kr/m²,12,Lägenhet,617 kr/mån,4 tr,1939.0,...,"24 m², 1 rum",södermalm,https://www.booli.se/bostad/637321,1 903 kr/mån,0,0,0,0,0,1


In [ ]:

df.loc[~df['location'].isin(top_X_location), 'location'] = 'OTHER'

    ohe = OneHotEncoder(use_cat_names=True, )
    dummies = ohe.fit_transform(df[['location']])
    joblib.dump(ohe, '../data/location_one_hot_encoder.pkl') # save transformer

    df = df.drop('location', axis = 1)
    df = df.join(dummies)